![](https://i.imgur.com/qkg2E2D.png)

# UnSupervised Learning Methods

## Exercise 004 - Part IV

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 17/06/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_03/Exercise0004Part004.ipynb)

In [ ]:
# !pip install umap-learn

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, MDS, TSNE
from sklearn.metrics import pairwise_distances
# from umap import UMAP
from umap.umap_ import UMAP

# Computer Vision

# Miscellaneous
import os
import math
from platform import python_version
import random
import time
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout


## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Import time module
import time

# record start time
start = time.time()

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

DATA_FILE_URL   = r'https://drive.google.com/uc?export=download&confirm=9iBg&id=1UXLdZgXwClgwZVszRq88UaaN2nvgMFiC'
DATA_FILE_NAME  = r'BciData.npz'


In [ ]:
# Auxiliary Functions

def Plot2DScatter(mX: np.ndarray, hA: plt.Axes, vC: np.ndarray = None) -> None:
    m = mX.min()
    M = mX.max()
    if vC is not None:
        hA.scatter(*mX.T, s = 50,  c = vC, edgecolor = 'k', alpha = 1)
    else:
        hA.scatter(*mX.T, s = 50,  c = 'lime', edgecolor = 'k', alpha = 1)
    hA.set_xlim([m, M])
    hA.set_ylim([m, M])
    hA.set_xlabel('$x_1$')
    hA.set_ylabel('$x_2$')


def PlotLabelsHistogram(vY: np.ndarray, hA = None, lClass = None, xLabelRot: int = None) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = (8, 6))

    vLabels, vCounts = np.unique(vY, return_counts = True)

    hA.bar(vLabels, vCounts, width = 0.9, align = 'center')
    hA.set_title('Histogram of Classes / Labels')
    hA.set_xlabel('Class')
    hA.set_ylabel('Number of Samples')
    hA.set_xticks(vLabels)
    if lClass is not None:
        hA.set_xticklabels(lClass)

    if xLabelRot is not None:
        for xLabel in hA.get_xticklabels():
            xLabel.set_rotation(xLabelRot)

    return hA


## Guidelines

 - Fill the full names and ID's of the team members in the `Team Members` section.
 - Answer all questions / tasks within the Jupyter Notebook.
 - Use MarkDown + MathJaX + Code to answer.
 - Verify the rendering on VS Code.
 - Submission in groups (Single submission per group).
 - You may and _should_ use the forums for questions.
 - Good Luck!

* <font color='brown'>(**#**)</font> The `Import Packages` section above imports most needed tools to apply the work. Please use it.
* <font color='brown'>(**#**)</font> You may replace the suggested functions to use with functions from other packages.
* <font color='brown'>(**#**)</font> Whatever not said explicitly to implement maybe used by a 3rd party packages.
* <font color='brown'>(**#**)</font> The total run time of this notebook must be **lower than 90 [Sec]**.

## Team Members

- Nadav_Talmon_203663950
- Nadav_Shaked_312494925
- Adi_Rosenthal_316550797

## Generate / Load Data

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if (DATA_FILE_NAME != 'None') and (not os.path.exists(DATA_FILE_NAME)):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

## 9. T-SNE and UMAP

In this section we'll use the _t-SNE_ and _UMAP_ algorithms to analyze EEG Signals in teh context of _Brain Computer Interface_ (BCI).  

### The BCI Data Set

The Brain Computer Interfaces (BCI) Data from [BCI Competition IV](https://www.bbci.de/competition/iv/).  
Specifically we'll use [data set 2a](https://www.bbci.de/competition/iv/#dataset2a) provided by the Institute for Knowledge Discovery (Laboratory of Brain Computer Interfaces), Graz University of Technology, (Clemens Brunner, Robert Leeb, Gernot Müller-Putz, Alois Schlögl, Gert Pfurtscheller).  
This is a recording of EEG signals while the subject is doing a cued movement of the left hand, right hand, feet or tongue.

The data is composed of:

 * 22 EEG channels (0.5-100Hz; notch filtered).
 * Sampling Rate of 250 [Hz] for 4 [Sec] for total of 1000 samples.
 * 4 classes: _left hand_, _right hand_, _feet_, _tongue_.
 * 9 subjects.
 * 287 measurements.

</br>

* <font color='brown'>(**#**)</font> You should install [UMAP](https://umap-learn.readthedocs.io/en/latest/index.html) on your system.

In [ ]:
# Generate Data

dData = np.load(DATA_FILE_NAME)
mX    = dData['mX1']
vY    = dData['vY1']

# Sample: An observation of a subject.
# Measurement: Sample in time of the EEG signal.
# Channel: EEG Channel.
numSamples, numMeasurements, numChannels = mX.shape
lLabel = ['Left Hand', 'Right Hand', 'Foot', 'Tongue'] #<! The labels

print(f'The data shape: {mX.shape}')
print(f'The number of samples: {mX.shape[0]}')
print(f'The number of measurements per sample: {mX.shape[1]}')
print(f'The number of EEG channels: {mX.shape[2]}')
print(f'The classes labels: {np.unique(vY)}')

* <font color='brown'>(**#**)</font> In the above `sample` is used per object detected.
* <font color='brown'>(**#**)</font> In the above `measurement` is used to describe the time sample (Like in Signal Processing).

In [ ]:
# Plot the Data

numRowsPlot = 3
numColsPlot = 2

numPlots = numRowsPlot * numColsPlot

hF, hAs = plt.subplots(nrows = numRowsPlot, ncols = numColsPlot, figsize = (18, 10))
hAs = hAs.flat

vIdx = np.random.choice(numSamples, numPlots, replace = False)

for sampleIdx, hA in zip(vIdx, hAs):
    mXX = mX[sampleIdx, :, :]
    hA.plot(mXX)
    hA.set_title(f'EEG Signals of Sample {sampleIdx:04d} with Label {lLabel[vY[sampleIdx]]}')
    hA.set_xlabel('Measurement Index')
    hA.set_ylabel('Measurement Value')

hF.tight_layout()

plt.show()

In [ ]:
# Plot the Data
# Show a shifted and scaled channels at a single plot.
# You may run it several times to observe different measurements.

numSamples, numMeasurements, numChannels = mX.shape #<! Samples, Time Samples, Channels
idx     = np.random.randint(numSamples)
mXi     = mX[idx, :, :].copy()
yi      = vY[idx]

# Normalizing and Vertically Shifting
mXi -= mXi.mean(0)
mXi /= 20
mXi += np.arange(numChannels)[None, :]
vT   = np.linspace(0, 4, numMeasurements, endpoint = False)

hF, hA = plt.subplots(figsize = (10, 6))
hA.plot(vT, mXi)
hA.set_title (f'Raw EEG Data\nTrue Label = {lLabel[yi]}')
hA.set_xlabel('Time [Sec]')

plt.show()

In [ ]:
# Histogram of Labels

hA = PlotLabelsHistogram(vY, lClass = lLabel)

* <font color='brown'>(**#**)</font> The data is balanced.

### 9.1. Processing RAW Data

In this section we'll compare 4 dimensionality reduction methods:

 - IsoMap.
 - MDS.
 - t-SNE.
 - UMAP.

We'll reduce the data dimension to `d = 2` and use the reference labels to display result.

The steps:

1. Data Pre Processing  
     * Reshape data into `mXRaw` such that each measurement is a row.  
       Namely the data should have dimensions of `287 x 22_000`.  
     * Apply dimensionality reduction using PCA to keep most of the data energy.
2. Set Parameters  
   Set parameters of each method to get results.  
   Set the amount of energy preserved using the PCA pre processing.
   **No need for grid search, just try and error of few values**.
3. Apply the Transform  
   Apply the transform on the data.  
   The low dimensional data should be in 2D space.  
4. Display Results  
   Create a subplot per method showing teh results and the used parameters.  

* <font color='brown'>(**#**)</font> No need to self implement any of the methods.

In [ ]:
#===========================Fill This===========================#
# 1. Pre Process data:
#    * Reshape to (287 x 22_000).
#    * Apply PCA to preserve `relEnergy` of the energy.
# 2. Apply Dimensionality Reduction (d = 2) using IsoMap, MDS, t-SNE and UMAP.
#    Choose a reasonable parameters for each.
# 3. Display the Low Dimensionality data.
#    Include the parameters in the title of each method.
# !! The output should be a figure of 1x4 axes (2D Scatter and 3D Scatter per method).
# !! You may use `Plot2DScatter()` for displaying the the data.

d = 2
relEnergy   = 0.95

mX_reshaped = mX.reshape(mX.shape[0], -1)

pca = PCA(n_components=relEnergy)
pca_mX = pca.fit_transform(mX_reshaped)
print("Number of dimensions to preserve desired relative energy:", pca_mX.shape[1])

algorithms = {
      'MDS': lambda : MDS(n_components=d, normalized_stress='auto'),
      'ISO map': lambda : Isomap(n_components=d, n_neighbors=5),
      't-SNE': lambda : TSNE(n_components=d),
      'UMAP': lambda : UMAP(n_components=d, n_neighbors=5)
}

fig, ax  = plt.subplots(1,4, figsize = (20, 6))

for i, (algorithm_name, algorithm) in enumerate(algorithms.items()):
    reduction = algorithm()
    mZ = reduction.fit_transform(pca_mX)
    Plot2DScatter(mZ, ax[i], vC = vY)
    ax[i].set_title(f"Method - {algorithm_name}, d = {d}")

plt.tight_layout()
plt.show()

#===============================================================#

### 9.1.1 Question

1. What's the purpose of the PCA pre process?
2. Explain the results. Think in the context of being able to classify the body part in action.

### 9.1.1 Solution

1. What's the purpose of the PCA pre process?

  The purpose of the PCA pre-processing step in this context is to reduce the dimensionality of the data while preserving most of its energy or variance.

  The PCA pre-processing can help to reduce computational complexity by reducing the dimensionality of the input data. It can remove noise or less relevant features from the data, and allowing to focus on the most important variations in the data.

  Note:
  PCA pre-processing not guaranteed good dimentionality reduction, and sometimes won't bring to a effective results, the results may vary depending on the dataset.

2. Explain the results. Think in the context of being able to classify the body part in action.

  By each of the dimentionality reduction method we can learn something for the pre-precessed PCA and the dimentionality reduction after it.

  By MDS technique we can learn that there is no similarity for each data label clasification group.

  By ISO-map technique we can learn that the data is not represented ny manifold

  By t-SNE technique we can learn that the local area for each data point is not related to the same classifcation

  Note:
  All of this assumption are related to results above (PCA pre-processing and then dimensionality reduction of each technique)

---

### 9.2. Feature Engineering

Form the above it is clear the RAW measurements are not a good representation of the data.  
Since each sample is basically $\boldsymbol{X}_{i}\in\mathbb{R}^{1000 \times 22}$ we can work on the time axis of the data.  

The strategy is as following:

 * Use the Covariance Matrix is a feature of the sample.
 * Use a metric optimized to covariance matrices.

Do the following steps:

1. Implements `GenCovMat()`  
   For each sample $\boldsymbol{X}_{i}\in\mathbb{R}^{1000 \times 22}$, compute the covariance matrix $\boldsymbol{C}_{i}\in\mathbb{R}^{22\times22}$.  
   You may use [`np.cov()`](https://numpy.org/doc/stable/reference/generated/numpy.cov.html).
2. Implement `SpdMetric()`  
   Given 2 SPD matrices in the form of vectors computes the geodesic distance between them.
3. Apply the Feature Engineering  
   The end of this process should be a matrix `mZ` with dimensions (287 x 22^2)
4. Generate the Distance Matrix  
   Using `SpdMetric()` generate the distance matrix of the data.  
   The output should be a matrix `mD` with dimensions (287 x 287).

#### The SPD Matrices Metric

An optimized metric for the manifold of SPD Matrices, is given by the SPD Metric (`SpdMetric`):

$$d\left(\boldsymbol{C}_{i},\boldsymbol{C}_{j}\right)=\sqrt{\sum_{i=1}^{d}\log^{2}\left(\lambda_{i}\left(\boldsymbol{C}_{i}^{-1}\boldsymbol{C}_{j}\right)\right)}$$

Where ${\lambda}_{i} \left( \cdot \right)$ extract the $i$ -th eigen value of the matrix.

In [ ]:
def GenCovMat(mX: np.ndarray) -> np.ndarray:
    '''
    Calculates the covariance matrices of the input data.
    Args:
        mX - Input data with shape (N x T x C)
    Output:
        mC - N covariance matrices with shape (N x C x C)
    '''

    N , T, C = mX.shape
    mC = np.zeros((N,C,C))

    for i in range(N):
        mC[i] = np.cov(mX[i].T)

    return mC

In [ ]:
def SpdMetric(vP: np.ndarray, vQ: np.ndarray) -> float:
    '''
    Calculates the AIRM geodesic distance between two SPD matrices.
    Args:
        mP    - An SPD matrix in the form of a vector with shape (d^2, )
        mQ    - An SPD matrix in the form of a vector with shape (d^2, )
    Output:
        geoDist - The geodesic distance, dist ≥ 0
    '''

    #===========================Fill This===========================#
    # 1. Convert the vectors into a matrix.
    # 2. Calculate the geodesic distance of SPD matrices.
    # !! Do not use any loop!
    # !! You may find `scipy.linalg.eigvalsh()` useful.

    d = int(np.sqrt(len(vP)))
    mC_i = vP.reshape((d, d))
    mC_j = vQ.reshape((d, d))

    eigenvalues = sp.linalg.eigvalsh(mC_j, mC_i)
    geoDist = np.sqrt(np.sum(np.log(eigenvalues) ** 2))

    #===============================================================#

    return geoDist

In [ ]:
# Process Data

#===========================Fill This===========================#
# 1. Generate the covariance matrices.
# 2. Reshape data into the samples matrix.
# 3. Calculate the distance matrix.
# !! Don't use any loop!
# !! You may use `pairwise_distances()` in order to avoid loops.

mC = GenCovMat(mX)

mC = mC.reshape(mC.shape[0], -1)
mD = pairwise_distances(mC, metric=SpdMetric)

In [ ]:
# Plot the Low Dimensional Data
# This shows the result using IsoMap with SpdMetric.

oIsoMapDr = Isomap(n_neighbors = 3, n_components = d, metric = 'precomputed')

hF = plt.figure(figsize = (12, 6))

mZi = oIsoMapDr.fit_transform(mD)
hA = hF.add_subplot()
Plot2DScatter(mZi, hA, vY)
hA.set_title(f'IsoMap (SPD Metric) with d = {d}')

plt.tight_layout()
plt.show()

#### 9.2.1. Question

In the above the _IsoMap_ is used.  
If one would like to use SciKit Learn's _Spectral Embedding_ (See [`SpectralEmbedding`](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.SpectralEmbedding.html)) one need to supply _Affinity Matrix_ (See `affinity` parameter).  
Describe how would you generate such matrix based on `SpdMetric()`.

* <font color='brown'>(**#**)</font> SciKit Learn's _Spectral Embedding_ is equivalent to _Laplacian Eigenmaps_ on lecture notes.

#### 9.2.1. Solution

To generate the affinity matrix for SciKit Learn's Spectral Embedding based on the SpdMetric(), you can follow these steps:

Calculate the distance matrix `mD` based on the SpdMetric() for your dataset. The distance matrix represents the pairwise geodesic distances between the covariance matrices.

Convert the distance matrix `mD` into the affinity matrix `m_aff` by applying the exponential function element-wise:

`m_aff[i, j] = np.exp(-mD[i, j])`.

The resulting affinity matrix `m_aff` is now suitable for use as the input affinity parameter in SciKit Learn's Spectral Embedding. It captures the pairwise similarities or dissimilarities between the covariance matrices based on the geodesic distances.

---

### 9.3. Dimensionality Reduction and Classification on Engineered Data

In this section we'll focus on `t-SNE` and `UMAP`.  

1. Apply Dimensionality Reduction  
   Apply `t-SNE` and / or `UMAP` on the engineered data.  
   Make sure to use the appropriate options to utilize the metric.  
   Apply this on the whole data.
2. Apply Classification  
   Apply a classifier (From SciKit Learn without Neural Based) on the low dimensional data.
3. Measure Performance of the Classifier
   Apply leave one out cross validation with score based on accuracy.  
   You should use [`cross_val_predict()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html).  
   This will generate a vector of 287 predictions.
4. Display Results  
   Display the confusion matrix and the total accuracy.

**This is a competition**:

 * The group with the 1st highest score will get 4 bonus points.
 * The group with the 2nd highest score will get 2 bonus points.


In [ ]:
#===========================Fill This===========================#
# 1. Use t-SNE and/or UMAP for Dimensionality Reduction.
# 2. On the output of (1) apply a classifier from SciKit Learn.
# 3. To measure the performance use `cross_val_predict()` with accuracy as score.
# 4. Display the confusion matrix of the result of the `cross_val_predict()`.
# !! Use the `SpdMetric()` for UMAP / t-SNE!
# !! This is a competition, optimize the hyper parameters of each step for best accuracy.
# !! No use of Neural Net based classifiers!
# !! If you use t-SNE, make sure to make it non random (See `init` and `random_state`).
# !! Pay attention to the run time limitation of the whole notebook.

from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import LeaveOneOut

d_umap = 4
algorithms = {
      'UMAP': lambda : UMAP(n_components=d_umap, negative_sample_rate=2, min_dist=1, metric=SpdMetric, random_state=seedNum)
}

algorithms_accuracy = {}

classifier = SVC(random_state=seedNum, gamma='auto')
pipeline = Pipeline([('classifier', classifier)])

loo = LeaveOneOut()

for algorithm_name, algorithm in algorithms.items():
    reduction = algorithm()
    mZ = reduction.fit_transform(mC)
    y_pred = cross_val_predict(pipeline, mZ, vY, cv=loo)
    algorithms_accuracy[algorithm_name] = accuracy_score(vY, y_pred)

    confusion_matrix_tsne = confusion_matrix(vY, y_pred, labels = np.unique(vY))
    plt.figure(figsize=(8, 6))
    plt.imshow(confusion_matrix_tsne, cmap='Blues')
    plt.title(f'Confusion Matrix - {algorithm_name}')
    plt.colorbar()
    plt.show()

#===============================================================#

In [ ]:
print(f"Accuracy UMAP = {algorithms_accuracy['UMAP'] * 100}%")

In [ ]:
end = time.time()
print(end - start)
assert(end - start <= 90)

> Royi: Great idea to check this! I will embrace.

#### 9.3.1. Question

In the above we had the whole data for the _dimensionality reduction_ step.  
Assume we use a method without _out of sample extension_.  
In the case above, how would you handle a new data point? Explain the pipeline.

#### 9.3.1. Solution

To handle a new data point in the case where we use a dimensionality reduction method without out-of-sample extension, we can follow the following pipeline:

Preprocess the new data point:

Use dimensionality reduction algorithm to transform the known data points into the reduced-dimensional space.
This involves applying the learned mapping or transformation function from the original feature space to the reduced-dimensional space.

Train a NN model that will get as input data point in the original space, and predicte data point after dimensionality reduction.

We should train this model by the training set we generate by the dimensionality reduction algorithm.
Classify the new data point:

By following this pipeline, we can incorporate the dimensionality reduction technique to handle dimensionality reduction for new data points.

---